## 1. Datalink

https://unhnewhaven-my.sharepoint.com/:f:/g/personal/vswar1_unh_newhaven_edu/EuRUHV4bpD1DncRghBdzXNMBX4SgJQ-et7JCtiihFo2vLQ?e=8MdnUp

## 2. Either create a new custom dataset class or adapt the data loading in the pytorch code for your dataset.

In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [39]:
class CustomRustDataset(Dataset):
    def __init__(self, img_dir, label_dir, classes, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.classes = classes
        self.transform = transform
        self.img_files = os.listdir(img_dir)
        self.label_files = [os.path.join(label_dir, f"{os.path.splitext(f)[0]}.txt") for f in self.img_files]

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img_height, img_width = img.shape[:2]
        
        # Read and normalize polygon points
        label_path = self.label_files[idx]
        polygons = []
        keypoints = []
        with open(label_path, 'r') as f:
            for line in f:
                points = line.strip().split()
                polygon = []
                for point in points:
                    try:
                        x, y = map(float, point.split(','))
                        # Normalize and add to polygon list
                        polygon.append((x / img_width, y / img_height))
                    except ValueError:
                        print(f"Warning: Skipping malformed point '{point}' in file '{label_path}'")
                if polygon:  # Only add non-empty polygons
                    polygons.append(polygon)
                    keypoints.extend(polygon)  # Add each point as a keypoint
        
        if self.transform:
            transformed = self.transform(image=img, keypoints=keypoints)
            img = transformed["image"]
            transformed_keypoints = transformed["keypoints"]

            # Reconstruct polygons from transformed keypoints
            polygons = []
            point_index = 0
            for polygon in polygons:
                polygon_points = transformed_keypoints[point_index:point_index + len(polygon)]
                polygons.append(polygon_points)
                point_index += len(polygon)

        labels = torch.tensor([0] * len(polygons), dtype=torch.int64)  # Single class for all polygons
        return img, labels, polygons


## 3 Data Partition

We have already split the dataset into Train, Val, and Test folders.

## 4.Normalize and augment according to the selected paper.

In [35]:
train_transforms = A.Compose([
    A.Resize(640, 640),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.HorizontalFlip(p=0.5),
    ToTensorV2()
], keypoint_params=A.KeypointParams(format='xy'))

val_transforms = A.Compose([
    A.Resize(640, 640),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], keypoint_params=A.KeypointParams(format='xy'))


In [40]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

train_data = CustomRustDataset(
    img_dir='C:/Program Files/Python311/RoadDamageDetector/images/train', 
    label_dir='C:/Program Files/Python311/RoadDamageDetector/labels/train', 
    classes=['Crevice Corrosion', 'Filiform Corrosion', 'Uniform Corrosion'], 
    transform=train_transforms
)

val_data=CustomRustDataset(
    img_dir='C:/Program Files/Python311/RoadDamageDetector/images/val', 
    label_dir='C:/Program Files/Python311/RoadDamageDetector/labels/val', 
    classes=['Crevice Corrosion', 'Filiform Corrosion', 'Uniform Corrosion'], 
    transform=val_transforms
)

test_data = CustomRustDataset(
    img_dir='C:/Program Files/Python311/RoadDamageDetector/images/test', 
    label_dir='C:/Program Files/Python311/RoadDamageDetector/labels/test', 
    classes=['Crevice Corrosion', 'Filiform Corrosion', 'Uniform Corrosion'], 
    transform=val_transforms
)


In [14]:
!pip install yolov5

     ---------------------------------------- 0.0/87.2 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/87.2 kB ? eta -:--:--
     ------------- ------------------------ 30.7/87.2 kB 435.7 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/87.2 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 87.2/87.2 kB 545.7 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/953.5 kB ? eta -:--:--
   ------ --------------------------------- 143.4/953.5 kB 2.8 MB/s eta 0:00:01
   ----------- ---------------------------- 276.5/953.5 kB 3.4 MB/s eta 0:00:01
   ----------------- ---------------------- 419.8/953.5 kB 3.3 MB/s eta 0:00:01
   ------------------------ --------------- 593.9/953.5 kB 3.1 MB/s eta 0:00:01
   ------------------------------- -------- 747.5/953.5 kB 3.4 MB/s eta 0:00:01
   -------------------------------------- 

  You can safely remove it manually.

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 5. Modify the Model’s Last Layer

In [18]:
import torch
from yolov5.models.yolo import Model

# Define the path to the configuration file
config_path = "C:/Program Files/Python311/RoadDamageDetector/yolov5/models/yolov5s.yaml"

# Load the model with the updated configuration file
model = Model(cfg=config_path)
model.to('cuda' if torch.cuda.is_available() else 'cpu')




                 from  n    params  module                                  arguments                     
  0                -1  1      3520  yolov5.models.common.Conv               [3, 32, 6, 2, 2]              
  1                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  2                -1  1     18816  yolov5.models.common.C3                 [64, 64, 1]                   
  3                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  4                -1  2    115712  yolov5.models.common.C3                 [128, 128, 2]                 
  5                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]              
  6                -1  3    625152  yolov5.models.common.C3                 [256, 256, 3]                 
  7                -1  1   1180672  yolov5.models.common.Conv               [256, 512, 3, 2]              
  8                -1  1   1182720  

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
     

In [20]:
import torch
from yolov5.models.yolo import Model
from yolov5.utils.loss import ComputeLoss

# Define the path to the configuration file
config_path = "C:/Program Files/Python311/RoadDamageDetector/yolov5/models/yolov5s.yaml"

# Define hyperparameters
hyp = {
    'cls': 0.5,
    'obj': 1.0,
    'iou_t': 0.2,
    'fl_gamma': 0.0,
    'cls_pw': 1.0,
    'obj_pw': 1.0,
    'anchor_t': 4.0,
    'giou': 0.05,
    'xy': 0.05,
    'wh': 0.05,
    'lr0': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005
}

# Load the model and set hyp
model = Model(cfg=config_path)
model.hyp = hyp  # Assign hyp to the model
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize ComputeLoss with the model
compute_loss = ComputeLoss(model)



                 from  n    params  module                                  arguments                     
  0                -1  1      3520  yolov5.models.common.Conv               [3, 32, 6, 2, 2]              
  1                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  2                -1  1     18816  yolov5.models.common.C3                 [64, 64, 1]                   
  3                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  4                -1  2    115712  yolov5.models.common.C3                 [128, 128, 2]                 
  5                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]              
  6                -1  3    625152  yolov5.models.common.C3                 [256, 256, 3]                 
  7                -1  1   1180672  yolov5.models.common.Conv               [256, 512, 3, 2]              
  8                -1  1   1182720  

## 6: Evaluate Performance on Training and Validation Data

In [41]:
import torch
from torch.utils.data import DataLoader

# Assuming `train_data` and `val_data` are already defined as subsets of the dataset

# Set up DataLoaders for batch processing
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_data, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Forward propagation function
def forward_propagation(dataloader, model):
    model.eval()  # Set model to evaluation mode
    all_predictions = []  # To store all predictions

    with torch.no_grad():  # Disable gradient calculations
        for images, _, _ in dataloader:
            images = torch.stack(images).to(device)
            predictions = model(images)
            all_predictions.append(predictions)

    return all_predictions

# Perform forward propagation on training and validation data
train_predictions = forward_propagation(train_loader, model)
val_predictions = forward_propagation(val_loader, model)

print("Forward propagation completed on training and validation datasets.")


Forward propagation completed on training and validation datasets.
Forward propagation completed on training and validation datasets.


## 7. Calculate average loss for training and validation data

In [42]:
train_loss = evaluate_model(train_loader, compute_loss)
val_loss = evaluate_model(val_loader, compute_loss)
print(f"Training Loss: {train_loss:.4f}")
print(f"Validation Loss: {val_loss:.4f}")


TypeError: ComputeLoss.__call__() takes 3 positional arguments but 4 were given

## 8. Visualization

In [43]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

def visualize_predictions(dataloader):
    model.eval()
    with torch.no_grad():
        for images, labels, polygons in dataloader:
            images = torch.stack(images).to(device)
            predictions = model(images)

            fig, axes = plt.subplots(1, len(images), figsize=(15, 15))
            for i, (image, pred_polygons) in enumerate(zip(images, predictions)):
                img_np = image.permute(1, 2, 0).cpu().numpy()
                axes[i].imshow(img_np)
                
                for polygon in polygons[i]:  # Draw each polygon
                    scaled_polygon = [(x * img_np.shape[1], y * img_np.shape[0]) for x, y in polygon]
                    poly_patch = Polygon(scaled_polygon, closed=True, edgecolor='r', facecolor='none', linewidth=2)
                    axes[i].add_patch(poly_patch)

            plt.show()
            break  # Show only the first minibatch

visualize_predictions(train_loader)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
